In [1]:
# Full reset of the camera
!echo 'dlinano' | sudo -S systemctl restart nvargus-daemon && printf '\n'
# Check device number
!ls -ltrh /dev/video*

# USB Camera (Logitech C270 webcam)
from jetcam.usb_camera import USBCamera
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

# CSI Camera (Raspberry Pi Camera Module V2)
# from jetcam.csi_camera import CSICamera
# camera = CSICamera(width=224, height=224)

camera.running = True
print("camera created")

/bin/bash: sudo: command not found
crw-rw---- 1 root video 81, 0 Feb 13 12:06 /dev/video0
camera created


In [2]:
!pip3 install jupyter_bbox_widget

In [3]:
# Set up the data directory location if not there already
DATA_DIR = '/nvdli-nano/data/object-detection/'
!mkdir -p {DATA_DIR}

In [7]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from dataset import ImageCollect

# initialize active dataset
directory = DATA_DIR
dataset = ImageCollect(directory)

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')

# manually update counts at initialization
count_widget.value = dataset.get_count()

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value)
    count_widget.value = dataset.get_count()
save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), count_widget, save_widget
])

# display(data_collection_widget)
print("data_collection_widget created")

NameError: name 'camera' is not defined

In [1]:
from jupyter_bbox_widget import BBoxWidget
import os
import base64

In [4]:
path = DATA_DIR
files = sorted(os.listdir(DATA_DIR))

annotations = {}
annotations_path = 'annotations.json'

In [5]:
def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

In [8]:
# a progress bar to show how far we got
w_progress = ipywidgets.IntProgress(value=0, max=len(files), description='Progress')
# the bbox widget
w_bbox = BBoxWidget(
    image = encode_image(os.path.join(path, files[0])),
    classes=['apple', 'orange', 'pear']
)

# combine widgets into a container
w_container = ipywidgets.VBox([
    w_progress,
    w_bbox,
])

In [9]:
# when Skip button is pressed we move on to the next file
def skip():
    w_progress.value += 1
    # open new image in the widget
    image_file = files[w_progress.value]
    w_bbox.image = encode_image(os.path.join(path, image_file))
    # here we assign an empty list to bboxes but 
    # we could also run a detection model on the file
    # and use its output for creating inital bboxes
    w_bbox.bboxes = [] 

# when Submit button is pressed we save current annotations
# and then move on to the next file
def submit():
    image_file = files[w_progress.value]
    # save annotations for current image
    annotations[image_file] = w_bbox.bboxes
    with open(annotations_path, 'w') as f:
        json.dump(annotations, f, indent=4)
    # move on to the next file
    skip()
    
w_bbox.on_skip(skip)
w_bbox.on_submit(submit)

<function __main__.submit()>

In [10]:
# display(data_collection_widget)

In [10]:
w_container